In [10]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import gc
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss

In [11]:
!pip3 install tensorflow_decision_forests

In [12]:
import tensorflow_decision_forests as tfdf

In [13]:
tfdf.keras.get_all_models()

[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel]

In [14]:
model = tfdf.keras.GradientBoostedTreesModel()
?model

In [16]:
sample_submission = pd.read_csv(r'/mnt/c/Users/Gebruiker/Desktop/Github/tensorflow/submission.csv')
train = pd.read_csv(r'/mnt/c/Users/Gebruiker/Desktop/Github/tensorflow/train.csv')
test = pd.read_csv(r'/mnt/c/Users/Gebruiker/Desktop/Github/tensorflow/test.csv')

In [17]:
train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,...,0,0,0,0,0,0,2,0,0,Class_6
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,2,0,0,0,0,0,1,0,3,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,3,0,0,7,0,1,5,2,2,0,...,0,4,0,2,2,0,4,3,0,Class_8
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [18]:
test.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,200001,1,2,0,0,0,0,0,0,0,...,3,1,3,0,0,0,0,3,0,0
2,200002,0,1,7,1,0,0,0,0,6,...,3,0,0,0,0,3,0,2,0,0
3,200003,0,0,0,4,3,1,0,0,0,...,0,0,0,1,0,0,0,4,0,0
4,200004,0,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
test_features = test.columns[1:]
train_features = train.columns[1:]
target = train['target'].values

In [20]:
train_oof = np.zeros((train.shape[0],9))
test_preds = 0
train_oof.shape

(200000, 9)

In [21]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test[test_features])


In [22]:
%%time

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_ds = train.loc[train_index]

    val_ds = train.loc[val_index]
    
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds[train_features], label="target")
    val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(val_ds[train_features], label="target")
    model = tfdf.keras.GradientBoostedTreesModel(num_trees=700, subsample = 0.9, max_depth = 3)
    model.fit(x=train_ds)
    val_pred = model.predict(val_ds)
    train_oof[val_index] = val_pred
    #print("Fold AUC:", roc_auc_score(val_target, val_pred[:,1]))
    test_preds += model.predict(test_ds)/n_splits
    del train_ds, val_ds
    gc.collect()

Fitting fold 1
2500/2500 [==============================] - 14s 4ms/step
Fitting fold 2
2500/2500 [==============================] - 11s 4ms/step
Fitting fold 3
2500/2500 [==============================] - 11s 4ms/step
Fitting fold 4
2500/2500 [==============================] - 11s 4ms/step
Fitting fold 5
2500/2500 [==============================] - 11s 4ms/step
CPU times: user 21min 37s, sys: 1min 13s, total: 22min 50s
Wall time: 6min 40s


In [23]:
oho = OneHotEncoder()
log_loss(oho.fit_transform(target.reshape(-1,1)).toarray(), train_oof)

1.7525331275527989

In [24]:
sample_submission[sample_submission.columns[1:]] = test_preds
sample_submission.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.045061,0.328445,0.157657,0.026780,0.014042,0.189828,0.031565,0.081530,0.125091
1,200001,0.042797,0.104225,0.067998,0.019020,0.013074,0.249680,0.074815,0.305041,0.123350
2,200002,0.029643,0.032905,0.029437,0.013594,0.008875,0.640774,0.033422,0.134264,0.077086
3,200003,0.060946,0.118672,0.080845,0.039256,0.016322,0.227183,0.075679,0.204227,0.176869
4,200004,0.042693,0.116489,0.083827,0.030996,0.015731,0.309619,0.056111,0.205265,0.139269
